<h1>Install the necessary packages</h1>

In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install numpy
!pip install munkres

<h1>Load the GRASP+PR algorithm</h1>

In [1]:
import numpy as np
from itertools import combinations
from sklearn.metrics.cluster import contingency_matrix
from munkres import Munkres
from scipy.spatial import distance


# This function returns the distance matrix for a dataset along with the sum of distances between every pair of points
def get_distances(data, ord=2):
  distances = distance.cdist(data, data, metric='minkowski', p=ord)
  weight = np.sum(distances) / 2
  return distances, weight


# This function calculates the minimum k-partition weight
def kcut(contributions, classes):
  return np.sum(contributions[classes, np.arange(contributions.shape[1])]) / 2


# This function updates the contribution matrix when moving a point from one cluster to another
def updated_contributions(pos, old_cluster, new_cluster, contributions, distances, k):
  contributions[old_cluster] -= distances[pos]
  contributions[new_cluster] += distances[pos]
  return contributions
  

# Code taken and modified from
# https://stackoverflow.com/questions/55258457/find-mapping-that-translates-one-list-of-clusters-to-another-in-python
# This function finds maps a clustering to an equivalent clustering nearest to another specified clustering
def translate_labels(master_list, convert_list):
  cont_mat = contingency_matrix(master_list, convert_list)
  munkres = Munkres()
  mapping = munkres.compute(cont_mat.max() - cont_mat)

  master_labels = np.unique(master_list)
  to_convert = np.unique(convert_list)

  map = {}
  for master_label, convert_label in mapping:
    map[to_convert[convert_label]] = master_labels[master_label]

  return map


# This function determines how different two clusterings are
def set_difference(classes, guiding):
  mapping = translate_labels(classes, guiding)
  guiding = [mapping[class_name] for class_name in guiding]

  diff = [pos for pos, (x, y) in enumerate(zip(classes, guiding)) if x != y]

  return len(diff) / len(guiding)

In [2]:
import numpy as np
import random
import time


# Code for the greedy build phase of the algorithm
def greedy_build(k, distances):
  assigned_clusters = [i for i in range(k)]
  contributions = distances[:k].copy()

  for index in range(k, distances.shape[0]):
    add_cluster = np.argmin(contributions[:, index])
    assigned_clusters.append(add_cluster)
    contributions[add_cluster] += distances[index]

  return np.array(assigned_clusters), contributions


# Code for the local search phase of the algorithm
def local_search(classes, k, contributions, distances):
  delta_weight = 0
  
  while True:
    local_best = contributions.argmin(axis=0)

    if (local_best == classes).all():
      break
      
    pos = np.argmax((local_best - classes) != 0)
      
    new_cluster = local_best[pos]
    cluster = classes[pos]
    classes[pos] = new_cluster

    delta_weight += contributions[new_cluster, pos] - contributions[cluster, pos]

    contributions = updated_contributions(pos, cluster, new_cluster, contributions, distances, k)

  return classes, contributions, delta_weight


# Code for the path relinking phase of the algorithm
def path_relinking(classes, k, elite_set, contributions, distances):
  guiding = random.choice(elite_set).copy()

  mapping = translate_labels(classes, guiding)
  guiding = [mapping[class_name] for class_name in guiding]

  diff = [pos for pos, (x, y) in enumerate(zip(classes, guiding)) if x != y]

  weight_delta = 0
  best_weight_delta = 0

  best_classes = classes.copy()
  best_contributions = contributions.copy()
  while len(diff) > 1:
    movements = [contributions[guiding[index], index] - contributions[classes[index], index] for index in diff]

    relink_index = np.argmin(movements)
    best = movements[relink_index]

    point_pos = diff[relink_index]

    cluster = classes[point_pos]
    new_cluster = guiding[point_pos]

    classes[point_pos] = guiding[point_pos]
    weight_delta += best

    contributions = updated_contributions(point_pos, cluster, new_cluster, contributions, distances, k)

    if weight_delta < best_weight_delta:
      best_weight_delta = weight_delta
      best_classes = classes.copy()
      best_contributions = contributions.copy()

    del diff[relink_index]

  return best_classes, best_contributions

In [3]:
from sklearn.metrics import rand_score
from scipy.spatial import distance
import numpy as np
import time
from sklearn.metrics import adjusted_rand_score

DEBUG = False
DEBUG_SEED = 0


def grasp_pr(data, iterations, k, max_elite, distances=None):
  REPLACE_THRESHOLD = 0.01

  best_weight = None
  best_solution = None
  elite_set = []
  elite_set_weights = []

  if distances is None:
    distances = distance.cdist(data, data, metric='minkowski', p=2)

  print('|', end='')

  checkpoint = 1

  if DEBUG:
    np.random.seed(DEBUG_SEED)
    random.seed(DEBUG_SEED)
    
  for i in range(iterations):
    updated_best = False
    
    shuffle_order = np.arange(distances.shape[0])
    np.random.shuffle(shuffle_order)

    shuffled_distances = distances[:, shuffle_order][shuffle_order]

    build, contributions = greedy_build(k, shuffled_distances)
    
    build, contributions, _ = local_search(build, k, contributions, shuffled_distances)
    
    weight = kcut(contributions, build)

    if i == 0:
      updated_best = False
      best_weight = weight

      unshuffle_order = np.zeros_like(shuffle_order)
      unshuffle_order[shuffle_order] = np.arange(distances.shape[0])
      unshuffled_build = build[unshuffle_order]

      best_solution = unshuffled_build
      elite_set += [unshuffled_build]
      elite_set_weights += [weight]

    else:
      unshuffle_order = np.zeros_like(shuffle_order)
      unshuffle_order[shuffle_order] = np.arange(distances.shape[0])
      unshuffled_build = build[unshuffle_order]
      unshuffled_contributions = contributions[:, unshuffle_order]

      if max_elite > 0:
        unshuffled_build, unshuffled_contributions = path_relinking(unshuffled_build, k, elite_set, unshuffled_contributions, distances)
  
        weight = kcut(unshuffled_contributions, unshuffled_build)
  
        if len(elite_set) < max_elite:
          elite_set += [unshuffled_build]
          elite_set_weights += [weight]
  
        else:
          if weight < min(elite_set_weights):
            worst = np.argmax(elite_set_weights)
            elite_set[worst] = unshuffled_build
            elite_set_weights[worst] = weight
  
          elif weight < max(elite_set_weights):
            difference = min([set_difference(unshuffled_build, elite_solution) for elite_solution in elite_set])
  
            if difference > REPLACE_THRESHOLD:
              worst = np.argmax(elite_set_weights)
              elite_set[worst] = unshuffled_build
              elite_set_weights[worst] = weight
      else:
        weight = kcut(unshuffled_contributions, unshuffled_build)

      if weight < best_weight:
        updated_best = True
        best_weight = weight

        best_solution = unshuffled_build

    # Prints the progress of the algorithm
    if (i + 1) >= checkpoint * (iterations // 100):
      print(checkpoint%10, end='')
      checkpoint += 1

  print()
  return best_solution, best_weight

<h1>Select a dataset to load</h1>

<h3>Iris</h3>

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

DATASET = 'datasets/iris.csv'

df = pd.read_csv(DATASET, index_col=False)

data = df.iloc[:, :4].values
distances, all_weights = get_distances(data)
output = df.iloc[:, 4].values

K = len(np.unique(output))

<h3>Palmer Penguins</h3>

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

DATASET = 'datasets/penguins.csv'

df = pd.read_csv(DATASET, index_col=[0])
numerical_data = df.get(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'year'])
categorical_data = df.get(['island', 'sex'])

num_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
numerical_data = num_imputer.fit_transform(numerical_data)

cat_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_data = cat_imputer.fit_transform(categorical_data)

encoder = OneHotEncoder()
categorical_data = encoder.fit_transform(categorical_data).toarray()

concatenated_data = np.append(numerical_data, categorical_data, axis=1)

scaler = StandardScaler()

data = scaler.fit_transform(concatenated_data)
distances, all_weights = get_distances(data)
output = df.get('species').values

K = len(np.unique(output))

<h3>MNIST</h3>

In [ ]:
from sklearn.metrics import adjusted_rand_score
from collections import Counter
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
import pandas as pd
import math
import numpy as np

size = 15000

# The subset of the MNIST dataset used was preprocessed and pickled for convenience
# due to its large size

with open(f'datasets/pickles/mnist{size}_data.npy', 'rb') as data_pickle:
  data = np.load(data_pickle)

with open(f'datasets/pickles/mnist{size}_output.npy', 'rb') as output_pickle:
  output = np.load(output_pickle)

distances, all_weights = get_distances(data)

K = len(np.unique(output))

<h3>Crop Recommendation</h3>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

DATASET = 'datasets/Crop_recommendation.csv'

df = pd.read_csv(DATASET, index_col=False)

data = df.iloc[:, :7].values
distances, all_weights = get_distances(data)
output = df.iloc[:, 7].values

K = len(np.unique(output))

<h3>Seeds</h3>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

DATASET = 'datasets/seeds_dataset.csv'

df = pd.read_csv(DATASET, index_col=None, header=None, sep='\t')

data = df.iloc[:, :7].values
distances, all_weights = get_distances(data)
output = df.iloc[:, 7].values

K = len(np.unique(output))

<h3>Leaf</h3>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

DATASET = 'datasets/leaf.csv'

df = pd.read_csv(DATASET, index_col=[1], header=None)

data = df.iloc[:, 1:].values
distances, all_weights = get_distances(data)
output = df.iloc[:, 0].values

K = len(np.unique(output))

<h3>Wine</h3>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

DATASET = 'datasets/wine.csv'

df = pd.read_csv(DATASET, index_col=None, header=None)

data = df.iloc[:, 1:].values
distances, all_weights = get_distances(data)
output = df.iloc[:, 0].values

K = len(np.unique(output))

<h3>G-set</h3>

In [ ]:
# Only the G1, G2, G3, G14, G15, G16, G22, G23, G24, G35, G36, G37, G43, G44, G45, G48, G49, and G50 graphs are included

def load_G(num):
  global distances, all_weights
  with open(f'datasets/Gset/G{num}', 'r') as f:
    n, lines = map(int, f.readline().split())
  
    distances = np.zeros((n, n), dtype=np.int32)
    for _ in range(lines):
      a, b, weight = map(int, f.readline().split())
      a -= 1; b -= 1
      distances[a, b] = weight
      distances[b, a] = weight

  all_weights = np.sum(distances) / 2

print('Load G_:')
g_val = int(input())
load_G(g_val)
data = None
K = 2

<h1>Run the algorithm</h1>

In [5]:
from sklearn.metrics import adjusted_rand_score
from collections import Counter
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
import pandas as pd
import math

MAX_ELITE = 10
iterations = 100

solution, min_k_partition_weight = grasp_pr(data, iterations, K, MAX_ELITE, distances=distances)
max_k_cut_weight = all_weights - min_k_partition_weight

print(f'Solution has a max k-cut weight of: {max_k_cut_weight}')
print(f'Solution obtains an adjusted Rand index of: {adjusted_rand_score(solution, output)}')

|1234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890
Solution has a max k-cut weight of: 193629.9696057929
Solution obtains an adjusted Rand index of: 0.5396619663398416
